In [ ]:
!pip install -r'/content/drive/MyDrive/GreenFlashProject/requirements.txt'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Adding the folder where our functions.py is located to sys.path
import sys
sys.path.append('/content/drive/MyDrive/GreenFlashProject')


In [6]:
# required commands to execute after installing dependencies
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 17.9 MB/s eta 0:00:00
 

In [7]:
# Basic data management
import importlib
import pandas as pd
import numpy as np
# functions have been extracted to model_comparison_functions1.py file
import models_comparison_functions1

from models_comparison_functions1 import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# this command allow us to force reload of our functions everytime we made some changes
# importlib.reload(models_comparison_functions1)

In [3]:
# Load dataframes
df_short_chats = pd.read_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_labeled_short_conv_dataset.csv')
df_large_chats = pd.read_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_labeled_large_convs_dataset.csv')
df_synthetic_chats = pd.read_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/all_conversations_synthetic.csv')

In [4]:
# lists to use with loops
datasets = [df_synthetic_chats, df_short_chats, df_large_chats]
dataset_names = ['df_synthetic_chats', 'df_short_chats', 'df_large_chats']

In [ ]:
# Some information about the datasets
for dataset in datasets:
  print(dataset.shape)
  print(dataset.columns)
  print(dataset.isna().sum())
  print(dataset.duplicated().sum())
  print(dataset['user_sentiment'].value_counts())

### Why fuctions were applied in diferent cells:

- Some models take a lot of time to process the three datasets, and after a couple of attempts and losing work due to execution environment disconnections, I decided to split the processing work into three parts.

In [ ]:
# Empty dataframe to store results
results_df = pd.DataFrame()

# TfidfVectorizer instance
vectorizer = TfidfVectorizer(max_features=5000)

# Iterate over all models aplying the functions in models_comparison_functions1.py
for i, dataset in enumerate(datasets):
    dataset_name = dataset_names[i]

    # Vectorizing texts
    X = vectorizer.fit_transform(dataset['user_clean_text'])

    # Defining labels
    y = dataset['user_sentiment']

    # ============ Data division ============ #
    # Some models use numerical labels - label encoder is included inside the function
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # =================== Apply to each model and add to results dataframe ==================== #
    # Asegurarse de que results_df se actualiza en cada paso sin sobrescribirlo

    # 1. VADER
    results_df = vader_sentiment_analysis(
        df=dataset, text_column='user_clean_text', target_column='user_sentiment',
        results_df=results_df, dataset_name=dataset_name
    )

    # 2. SVM
    results_df = SVM_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )

    # 3. Naive Bayes
    results_df = naive_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )

    # 4. Logistic Regression
    results_df = logistic_regression_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )

    # 5. Random Forest
    results_df = rforest_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )

    # 6. LightGBM (requires numerical labels)
    results_df = lightgbm_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )

    # 7. XGBoost (requires numerical labels)
    results_df = xgboost_sentiment(
        X_train, X_test, y_train, y_test, results_df=results_df, dataset_name=dataset_name
    )



# export final dataset
results_df.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/results_df.csv', index=False)

[codecarbon INFO @ 07:48:17] [setup] RAM Tracking...
[codecarbon INFO @ 07:48:17] [setup] GPU Tracking...
[codecarbon INFO @ 07:48:17] No GPU found.
[codecarbon INFO @ 07:48:17] [setup] CPU Tracking...
[codecarbon WARNING @ 07:48:17] No CPU tracking mode found. Falling back on CPU constant mode.


Input DataFrame shape: (1063, 7)
Columns in DataFrame: ['conversation_id', 'model', 'system_prompt', 'full_messages', 'user_clean_text', 'Assistant_messages', 'user_sentiment']


[codecarbon INFO @ 07:48:18] CPU Model on constant consumption mode: AMD EPYC 7B12
[codecarbon INFO @ 07:48:18] >>> Tracker's metadata:
[codecarbon INFO @ 07:48:18]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:48:18]   Python version: 3.10.12
[codecarbon INFO @ 07:48:18]   CodeCarbon version: 2.6.0
[codecarbon INFO @ 07:48:18]   Available RAM : 50.994 GB
[codecarbon INFO @ 07:48:18]   CPU count: 8
[codecarbon INFO @ 07:48:18]   CPU model: AMD EPYC 7B12
[codecarbon INFO @ 07:48:18]   GPU count: None
[codecarbon INFO @ 07:48:18]   GPU model: None


ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv


[codecarbon INFO @ 07:48:18] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 07:48:19] Energy consumed for RAM : 0.000001 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 07:48:19] Energy consumed for all CPUs : 0.000006 kWh. Total CPU Power : 120.0 W
[codecarbon INFO @ 07:48:19] 0.000007 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. U

...

[codecarbon INFO @ 11:05:57] [setup] RAM Tracking...
[codecarbon INFO @ 11:05:57] [setup] GPU Tracking...
[codecarbon INFO @ 11:05:57] No GPU found.
[codecarbon INFO @ 11:05:57] [setup] CPU Tracking...
[codecarbon WARNING @ 11:05:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:05:58] CPU Model on constant consumption mode: AMD EPYC 7B12
[codecarbon INFO @ 11:05:58] >>> Tracker's metadata:
[codecarbon INFO @ 11:05:58]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 11:05:58]   Python version: 3.10.12
[codecarbon INFO @ 11:05:58]   CodeCarbon version: 2.6.0
[codecarbon INFO @ 11:05:58]   Available RAM : 50.994 GB
[codecarbon INFO @ 11:05:58]   CPU count: 8
[codecarbon INFO @ 11:05:58]   CPU model: AMD EPYC 7B12
[codecarbon INFO @ 11:05:58]   GPU count: None
[codecarbon INFO @ 11:05:58]   GPU model: None


ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv


[codecarbon INFO @ 11:05:58] Saving emissions data to file /content/emissions.csv
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:05:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[codecarbon INFO @ 11:06:13] Energy consumed for RAM : 0.000080 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 11:06:13] Energy consumed for all CPUs : 0.000500 kWh. Total CPU Power : 120.0 W
[codecarbon INFO @ 11:06:14] 0.000580 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:06:28] Energy consumed for RAM : 0.000159 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 11:06:28] Energy consumed for all CPUs : 0.001000 kWh. Total CPU Power : 120.0 W
[codecarbon INFO @ 11:06:28] 0.001159 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:06:43] Energy consumed for RAM : 0.000239 kWh. RAM Power : 19.122615337371826 W
[codecarbon INFO @ 11:06:43] Energ

ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json
ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json


,Model,Dataset,Training Time (s),Original Prediction Time (s),Adjusted Prediction Time (s),Training Emissions (kg CO2e),Original Prediction Emissions (kg CO2e),Adjusted Prediction Emissions (kg CO2e),Accuracy,Precision,Recall,F1-Score
0,VADER,df_synthetic_chats,0.000000,0.170611,0.170611,0.000000e+00,4.348028e-06,4.348028e-06,0.000000,0.000000,0.000000,0.000000
1,SVM,df_synthetic_chats,0.828634,0.032706,0.163528,2.064772e-05,2.181290e-05,1.090645e-04,0.892019,0.895758,0.892019,0.892990
2,Naive Bayes,df_synthetic_chats,0.008471,0.001245,0.006225,3.075526e-07,6.766535e-07,3.383267e-06,0.887324,0.887432,0.887324,0.884736
3,Logistic Regression,df_synthetic_chats,0.030896,0.001547,0.007735,8.606232e-07,1.235712e-06,6.178560e-06,0.887324,0.890640,0.887324,0.887730
4,Random Forest,df_synthetic_chats,0.366300,0.008888,0.044441,9.174521e-06,9.759814e-06,4.879907e-05,0.920188,0.922232,0.920188,0.920021
5,LightGBM,df_synthetic_chats,0.877248,0.005737,0.028685,8.084905e-08,8.284752e-08,4.142376e-07,0.887324,0.886489,0.887324,0.886518
6,XGBoost,df_synthetic_chats,0.970652,0.004338,0.021689,2.419977e-05,2.468866e-05,1.234433e-04,0.901408,0.900731,0.901408,0.900868
7,VADER,df_short_chats,0.000000,68.625856,68.625856,0.000000e+00,1.700161e-03,1.700161e-03,0.000000,0.000000,0.000000,0.000000
8,SVM,df_short_chats,9892.311807,233.646207,1168.231037,2.455172e-01,2.513180e-01,1.256590e+00,0.860085,0.860800,0.860085,0.860286
9,Naive Bayes,df_short_chats,0.227476,0.005969,0.029845,5.705493e-06,6.280578e-06,3.140289e-05,0.764238,0.779839,0.764238,0.763695


In [11]:
# Empty dataframe
results_df2 = pd.DataFrame()

# Iterate over all datasets aplying RoBerta function
for i, dataset in enumerate(datasets):
    dataset_name = dataset_names[i]
    # 9. RoBERTa
    results_df2 = roberta_sentiment(dataset, text_column='user_clean_text', label_column='user_sentiment',
                                   results_df=results_df2, dataset_name=dataset_name)

  # Results into CSV
results_df2.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/results_df-2.csv', index=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
[codecarbon INFO @ 17:03:41] [setup] RAM Tracking...
[codecarbon INFO @ 17:03:41] [setup] GPU Tracking...
[codecarbon INFO @ 17:03:41] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:03:41] [setup] CPU Tracking...
[codecarbon WARNING @ 17:03:41] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:03:42] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:03:42] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 17:03:42] >>> Tracker's metadata:
[codecarbon INFO

ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv


[codecarbon INFO @ 17:03:42] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 17:03:51] Energy consumed for RAM : 0.000744 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:03:51] Energy consumed for all GPUs : 0.001702 kWh. Total GPU Power : 42.22360862714191 W
[codecarbon INFO @ 17:03:51] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:03:51] 0.004040 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:03:55] Energy consumed for RAM : 0.000071 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:03:55] Energy consumed for all GPUs : 0.000172 kWh. Total GPU Power : 48.143362477958206 W
[codecarbon INFO @ 17:03:55] Energy consumed for all CPUs : 0.000152 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:03:55] 0.000394 kWh of electricity used since the beginning.


ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
[codecarbon INFO @ 17:03:56] [setup] RAM Tracking...
[codecarbon INFO @ 17:03:56] [setup] GPU Tracking...
[codecarbon INFO @ 17:03:56] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:03:56] [setup] CPU Tracking...
[codecarbon WARNING @ 17:03:56] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:03:57] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:03:57] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 17:03:57] >>> Tracker's metadata:
[codecarbon INFO

...

ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json


[codecarbon INFO @ 17:29:51] Energy consumed for RAM : 0.009346 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:29:51] Energy consumed for all GPUs : 0.026333 kWh. Total GPU Power : 60.77907519940484 W
[codecarbon INFO @ 17:29:51] Energy consumed for all CPUs : 0.020007 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:29:51] 0.055686 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:30:06] Energy consumed for RAM : 0.001406 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:30:06] Energy consumed for all GPUs : 0.004276 kWh. Total GPU Power : 60.23769630956244 W
[codecarbon INFO @ 17:30:06] Energy consumed for all CPUs : 0.003010 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:30:06] 0.008692 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:30:06] Energy consumed for RAM : 0.009429 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:30:06] Energy consumed for all GPUs : 0.026585 kWh. Total GPU Power : 60.48465188161225

ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json


[codecarbon INFO @ 17:31:40] Energy consumed for RAM : 0.001923 kWh. RAM Power : 19.86054039001465 W
[codecarbon INFO @ 17:31:40] Energy consumed for all GPUs : 0.005850 kWh. Total GPU Power : 61.762229715551825 W
[codecarbon INFO @ 17:31:40] Energy consumed for all CPUs : 0.004117 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:40] 0.011890 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:31:40] 0.016053 g.CO2eq/s mean an estimation of 506.26085759778465 kg.CO2eq/year


ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json
ref: /usr/local/lib/python3.10/dist-packages/codecarbon/data/private_infra/global_energy_mix.json


In [ ]:
# Iterate over all datasets aplying the Spacy function
results_df = pd.DataFrame()
for i, dataset in enumerate(datasets):
    dataset_name = dataset_names[i]
    # 8. Spacy Sentiment (requiere el dataframe completo y etiquetas en texto)
    results_df = spacy_sentiment(
        dataset, text_column='user_clean_text', label_column='user_sentiment',
        results_df=results_df, dataset_name=dataset_name
    )

# Results into CSV
results_df.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/results_df-3.csv', index=False)



---


#Now we have three datasets with the metrics for each model to start analyzing performance


---

